In [1]:
# Setup
import sys
print(sys.version)
import numpy as np
import tensorflow as tf
from Utils.PokemonUtils import PokemonUtils as utils
from matplotlib import pyplot as plt
from Models import baseline

utils.loadAllGIFs()
x_train, x_val, x_test, y_train, y_val, y_test = utils.generateXYSplitsV1()

def trainBatches(batch_size):
    for batch in range(len(x_train) / batch_size):
        yield x_train[batch * batch_size: (batch + 1) * batch_size], y_train[batch * batch_size: (batch + 1) * batch_size]

def valBatches(batch_size):
    for batch in range(len(x_val) / batch_size):
        yield x_val[batch * batch_size: (batch + 1) * batch_size], y_val[batch * batch_size: (batch + 1) * batch_size]

def testBatches(batch_size):
    for batch in range(len(x_test) / batch_size):
        yield x_test[batch * batch_size: (batch + 1) * batch_size], y_test[batch * batch_size: (batch + 1) * batch_size]

2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 11:07:58) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


ImportError: No module named tensorflow

In [ ]:
def check_accuracy(sess, dset, x, scores):
    num_correct, num_samples = 0, 0
    for x_batch, y_batch in dset:
        feed_dict = {x: x_batch, is_training: 0}
        scores_np = sess.run(scores, feed_dict=feed_dict)
        y_pred = scores_np.argmax(axis=1)
        num_samples += x_batch.shape[0]
        num_correct += (y_pred == y_batch).sum()
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

In [ ]:
# Train model
model = baseline.ShallowNet()

num_epochs = 2
learning_rate = 1e-3
batch_size = 32

tf.reset_default_graph()    
with tf.device(device):
    x = tf.placeholder(tf.float32, [None, 128, 128, 4])
    y = tf.placeholder(tf.int32, [None])

    is_training = tf.placeholder(tf.bool, name='is_training')

    scores = model_init_fn(x, is_training)

    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
    loss = tf.reduce_mean(loss)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        t = 0
        for epoch in range(num_epochs):
            print('Starting epoch %d' % epoch)
            for x_batch, y_batch in trainBatches(batch_size):
                feed_dict = {x: x_batch, y: y_batch, is_training:1}
                loss_np, _ = sess.run([loss, train_op], feed_dict=feed_dict)
                if t % print_every == 0:
                    print('Iteration %d, loss = %.4f' % (t, loss_np))
                    check_accuracy(sess, valBatches(batch_size), x, scores)
                    print()
                t += 1
                    

In [ ]:
# Test model
acc = None
pass
print(acc)